In [1]:
import json
import nltk
import re
import string
from nltk.corpus import stopwords 
import demoji
from nltk.chunk import conlltags2tree, tree2conlltags
import spacy
stop_words = set(stopwords.words('english')) 
demoji.download_codes()

... OK (Got response in 0.28 seconds)
Writing emoji data to C:\Users\victo\.demoji/codes.json ...
... OK


In [2]:
with open('gg2020.json', encoding='utf8') as json_file:
    data = [json.loads(line) for line in json_file]

In [16]:
data = json.load(open('gg2013.json'))

In [4]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [3]:
def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [18]:
clean_data = []
count = 0
for tweet in data:
    tweet_text = tweet['text']
    no_http = re.sub('http://\S+|https://\S+', '', tweet_text)
    remove_tag = strip_all_entities(no_http)
    clean_data.append(remove_tag.lower())

In [19]:
import itertools
clean_data.sort()
clean_data = list(clean_data for clean_data,_ in itertools.groupby(clean_data))

In [20]:
# search host
host_list = ['host', 'hosts', 'hosting', 'hosted']
host_tweet = []
for tweet in clean_data:
    for test_word in host_list:
        if test_word in tweet:
            host_tweet.append(tweet)
            break

In [21]:
tag = nltk.pos_tag(clean_data[0])
pattern = 'NP: {<DT>?<JJ>*<NN>}'
cp = nltk.RegexpParser(pattern)
cs = cp.parse(tag)
iob = tree2conlltags(cs)

In [22]:
i = 0
nlp = spacy.load("en_core_web_sm")
#doc = nlp(' '.join(word for word in clean_data[i]))
#print(' '.join(word for word in clean_data[i]))
#for ent in doc.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
name_list = []
for tweet in host_tweet:
    doc = nlp(tweet)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name_list.append(ent.text)

In [13]:
unique_list = list(set(name_list))
counter = []
for item in unique_list:
    counter.append(name_list.count(item))
sort_counter = sorted(range(len(counter)), key=lambda k: counter[k], reverse=True)    

In [14]:
possible_list = []
for i in range(5):
    name = unique_list[sort_counter[i]]
    possible_list.append(name)
reduced_possible_list = []
for i in range(5):
    test = possible_list[i]
    conflict = 0
    for j in range(5):
        if test in possible_list[j] and i != j:
            conflict = 1
            break
    if test not in reduced_possible_list and conflict == 0:
        reduced_possible_list.append(test)
# search first
hosts = []
hosts.append(reduced_possible_list[0])
host_idx = possible_list.index(hosts[0])
max_num = counter[sort_counter[host_idx]]
# search second
name_pair = []
for name1 in reduced_possible_list:
    for name2 in reduced_possible_list:
        count = 0
        for tweet in host_tweet:
            tweetstr = ' '.join(map(str, tweet)) 
            if name1 + ' and ' + name2 in tweetstr or name2 + ' and ' + name1 in tweetstr:
                count += 1
        if count > max_num * 0.1:
            name_pair.append([name1, name2, count])
name_pair.sort(key=lambda x: x[2], reverse=True)
if len(name_pair) > 0:
    hosts = [name_pair[0][0],name_pair[0][1]]

In [15]:
print(hosts)

['Ricky Gervais']


#### Find awards

In [4]:
pattern = re.compile(r'(?<=w[oi]n[s]\s)[Bb]est.*')
tweet_sep = ['for','who',' s ','made','yet','?', 'golden', 'globe', 'globes',' | ', 'with' ,' at', ' http', ' #', '(','.', ',', '!', '?','\\', ':', ';', '"', "'",'the','but','although', 'made']
awards_bag_of_words = ['-','performance', 'actress', 'actor', 'supporting', 'role', 'director', 'motion', 'picture', 'drama','animated', 'feature', 'film', 'song','comedy', 'musical', 'language', 'foreign','screenplay', 'orginal', 'television', 'tv', 'series', 'mini-series', 'mini']


In [38]:
awards = []
for tweetx in data:
    tweet = tweetx['text']
    award = re.findall(pattern, tweet )
    
    if award:
        #print(' '.join(map(str, tweet)))
        #print(award)
        for a in award:
            #a = a.lower()
            for sep in tweet_sep:
                if sep in a:
                    a = a.split(sep)[0]
            if "/" in a:
                a = a.replace('/',' or ')
            awards.append(a)

In [34]:
print(awards)

['best personality on ', 'best movie', 'best dressed i m obsessed ', 'best facial hair of ', 'best supporting actor ', 'best supporting actor that', 'best supporting actor', 'best supporting', 'best supporting actor ', 'best supporting actor ', 'best supporting actor', 'best supporting actor ', 'best supporting actor ', 'best supporting actor did not think that was gon na happen', 'best supporting actor', 'best supporting actor ', 'best suppporting actor my favorite o', 'best supporting actor ', 'best supporting actor ', 'best supporting actor haven t seen django ', 'best supporting actor ', 'best supporting actor really enjoyed his per', 'best supporting actor motion picture ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actress in a miniseries ', 'best supporting actor', 'best supporting actress in a minis

In [35]:
valid_awards = []
for each in awards:
    each = each.lower()
    if len(set(each.split(" ")).intersection(set(awards_bag_of_words)))>2:
        valid_awards.append(each)

In [16]:
print(valid_awards)

['best supporting actor - motion picture ', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best supporting tv actress in downton abbey', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best mini-series or motion picture made ', 'best supporting actress in a tv show', 'best mini series or tv movie', 'best supporting actress in a tv show', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or motion picture made ', 'best actress in a tv series ', 'best actress in mini-series or tv movie ', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or motion picture made ', 'best actress in a mini series or tv movie', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or motion picture made ', 'best actress in a mini-series or tv movie ', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or m

In [10]:
!pip install fuzzywuzzy

In [15]:
from fuzzywuzzy import fuzz

def dataProcess(lawards):
    for (i,a) in enumerate(lawards):
        if "performance" not in a and ("actor" in a or "actress" in a) and "supporting" not in a:
            if "actor" in a:
                lawards[i] = a.replace("actor","performance by an actor")
            elif "actress" in a:
                lawards[i] = a.replace("actress","performance by an actress")
    return lawards

def remove_duplicate(all_data, limit):
    for i in range(0, len(all_data)):
        for j in range(i+1, len(all_data)):
            if fuzz.ratio(all_data[i], all_data[j]) > limit or fuzz.token_set_ratio(all_data[i], all_data[j]) > limit or fuzz.token_sort_ratio(all_data[i], all_data[j]) > limit or fuzz.partial_ratio(all_data[i], all_data[j]) > limit:
                all_data[j] = ""
    return list(set(all_data))

In [36]:
valid_awards.sort(key=lambda a: len(a),reverse=True)

valid_awards = [a[0] for a in nltk.FreqDist(valid_awards).most_common(100)]
#print(valid_awards)
valid_awards = remove_duplicate(valid_awards, 95)
valid_awards = dataProcess(valid_awards)
print(valid_awards)

['', 'best performance by an actor in a musical or comedy award', 'best performance by an actress tv series calls nominees in category brilliant', 'best animated feature film via', 'best performance by an actress in a drama series', 'best television show musical comedy congrats xoxoxoxo', 'best supporting actress in a motion picture ', 'best director motion picture ', 'best supporting actor motion picture ', 'best performance by an actor movie comedy or musical sing talk to me hugh', 'best performance by an actor amp takes best musical comedy good night', 'best supporting actor in a series or tv movie', 'best tv series drama', 'best performance by an actress comedy musical first win of ', 'best performance by an actress tv drama i want to thank ', 'best picture comedy or musical have to see it', 'best performance by an actress in a comedy series', 'best performance by an actor in a miniseries or motion picture ', 'best supporting actress in a tv show everyone playing tina and amy', 'be

In [37]:
for a_name in valid_awards:
    print(a_name)


best performance by an actor in a musical or comedy award
best performance by an actress tv series calls nominees in category brilliant
best animated feature film via
best performance by an actress in a drama series
best television show musical comedy congrats xoxoxoxo
best supporting actress in a motion picture 
best director motion picture 
best supporting actor motion picture 
best performance by an actor movie comedy or musical sing talk to me hugh
best performance by an actor amp takes best musical comedy good night
best supporting actor in a series or tv movie
best tv series drama
best performance by an actress comedy musical first win of 
best performance by an actress tv drama i want to thank 
best picture comedy or musical have to see it
best performance by an actress in a comedy series
best performance by an actor in a miniseries or motion picture 
best supporting actress in a tv show everyone playing tina and amy
best animated feature film wreck it ralph was a better movie


In [26]:
for key in ans['award_data']:
    print(key)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a supporting role in a motion picture
best television serie

In [110]:
awards_list = {}
for tweet in clean_data:
    if "the" in tweet and "award" in tweet:
        #print(tweet)
        collect = []
        for word in list(reversed(tweet)):  
            if word == "award":
                collect.insert(0,word)
            elif word == "the" and len(collect) > 0:
                #print(collect_str)
                collect_str = ' '.join(map(str, collect))
                if len(collect) > 2 and len(collect) <= 4 and 'golden' not in collect_str:
                    doc = nlp(collect_str)
                    for ent in doc.ents:
                        if ent.label_ == "PERSON":                    
                            if collect_str not in awards_list:
                                awards_list[collect_str] = 0
                            awards_list[collect_str] += 1
                            break
                        else:
                            break

                collect = []
            elif len(collect) > 0:
                collect.insert(0,word)

In [90]:
print(awards_list)

{'brightest genius award': 1, 'carol burnett award': 109, 'cecille b demille award': 1, 'carroll burnett award': 2, 'prestigious carol burnett award': 1, 'chairman s award': 1, 'carol burnett show award': 1, 'green goddess award': 1, 'event carol burnett award': 1, 'galton an award': 1, 'honorary carol burnett award': 1, 'carol burnet award': 1, 'inaugural carol burnett award': 1, 'jussie smollett award': 1, 'director s guild award': 1, 'official kickoff to award': 1}


In [ ]:
for tweet in clean_data:
    tweetstr = ' '.join(map(str, tweet)) 

In [24]:
ans = json.load(open('gg2013answers.json'))

In [ ]:
print